# Ensembl genes table extraction EDA

This notebook is useful for development as well as exploratory data analysis on the extracted tables.
It is currently automically executed and saved as part of exports using `papermill`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from bioregistry import normalize_prefix

from ensembl_genes import ensembl_genes

In [3]:
# parameters cell
species = "human"
release = "111"

In [4]:
# Parameters
species = "mouse"
release = "113"


In [5]:
ensg = ensembl_genes.Ensembl_Gene_Queries(release=release, species=species)
ensg.connection_url

'mysql+mysqlconnector://anonymous@ensembldb.ensembl.org:3306/mus_musculus_core_113_39'

In [6]:
database = ensg.database
database

'mus_musculus_core_113_39'

## Extract data

## gene attrib counts

In [7]:
ensg.run_query("gene_attrib_counts").head(15)

,attrib_type_id,code,name,description,attrib_type_count,attrib_type_examples
0,142,GeneGC,Gene GC,Percentage GC content for this gene,78298,"52.38, 41.29, 45.11, 46.08, 58.82, 45.96, 38.3..."
1,4,name,Name,Alternative/long name,72073,"RP24-408N5.1, Clec12b, RP23-454G7.1, Slc35e2, ..."
2,395,xref_id,Xref ID,ID of associated database reference,48559,"OTTMUSG00000050759, OTTMUSG00000056465, OTTMUS..."
3,538,legacy_biotype,Legacy biotype,Obsolete biotype previously assigned to this E...,32957,"antisense, lincRNA, sense_overlapping, process..."
4,54,remark,Remark,Annotation remark,23152,"TAGENE_gene, D1Ertd622e, (Dodds et al., J Biol..."
5,380,havana_cv,Havana CV term,Controlled vocabulary terms from Havana,11532,"overlapping locus, overlaps pseudogene, confir..."
6,382,NoTransRefError,No translations due to reference error,This gene is believed to include protein codin...,5,1
7,1,embl_acc,European Nucleotide Archive (was EMBL) accession,ENA,0,None
8,2,status,Status,None,0,None
9,3,synonym,Synonym,None,0,None


## genes

In [8]:
ensg.gene_df.head()

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
0,ENSMUSG00000000001,5,Gnai3,MGI,MGI:95773,protein_coding,ensembl_havana,2004-11-18 15:05:57,2020-08-10 14:30:16,GRCm39,...,108014596,108053462,-1,True,NaN,None,G protein subunit alpha i3,MGI Symbol,MGI:95773,ENSMUSG00000000001
1,ENSMUSG00000000003,16,Pbsn,MGI,MGI:1860484,protein_coding,ensembl_havana,2006-06-28 13:29:12,2020-08-10 14:24:33,GRCm39,...,76881507,76897229,-1,True,NaN,None,probasin,MGI Symbol,MGI:1860484,ENSMUSG00000000003
2,ENSMUSG00000000028,16,Cdc45,MGI,MGI:1338073,protein_coding,ensembl_havana,2007-06-26 09:54:29,2020-08-10 14:17:14,GRCm39,...,18599197,18630737,-1,True,NaN,None,cell division cycle 45,MGI Symbol,MGI:1338073,ENSMUSG00000000028
3,ENSMUSG00000000031,20,H19,MGI,MGI:95891,lncRNA,ensembl_havana,2006-02-02 11:27:21,2024-05-06 19:10:14,GRCm39,...,142129262,142133957,-1,True,NaN,None,"H19, imprinted maternally expressed transcript",MGI Symbol,MGI:95891,ENSMUSG00000000031
4,ENSMUSG00000000037,18,Scml2,MGI,MGI:1340042,protein_coding,ensembl_havana,2006-08-18 16:16:32,2020-08-10 14:22:47,GRCm39,...,159865521,160041209,1,True,NaN,None,Scm polycomb group protein like 2,MGI Symbol,MGI:1340042,ENSMUSG00000000037


In [9]:
# clone-based genes no longer get a symbol and are filled with the stable ID
# https://www.ensembl.info/2021/03/15/retirement-of-clone-based-gene-names/
ensg.gene_df.query("gene_symbol == ensembl_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
13860,ENSMUSG00000047061,7,ENSMUSG00000047061,None,None,TEC,ensembl_havana,2018-12-03 13:41:15,2020-08-10 14:05:17,GRCm39,...,45232168,45234698,1,True,NaN,None,TEC,NaN,NaN,ENSMUSG00000047061
20035,ENSMUSG00000072963,3,ENSMUSG00000072963,None,None,lncRNA,ensembl_havana,2018-11-30 11:32:45,2020-08-10 14:22:30,GRCm39,...,53345631,53347937,-1,True,NaN,None,novel transcript,NaN,NaN,ENSMUSG00000072963


In [10]:
# which external database the gene symbol derives from versus the ensembl source
pd.crosstab(
    ensg.gene_df.ensembl_source,
    ensg.gene_df.gene_symbol_source_db.fillna("missing (clone-based)"),
    margins=True,
)

gene_symbol_source_db,EntrezGene,HGNC,MGI,missing (clone-based),All
ensembl_source,,,,,
RefSeq,0,0,37,0,37
ensembl,3,0,4924,51,4978
ensembl_havana,35,0,22435,38,22508
havana,87,3,26270,454,26814
havana_tagene,165,0,1269,21340,22774
mirbase,0,0,1187,0,1187
All,290,3,56122,21883,78298


In [11]:
ensg.gene_df.coord_system.value_counts().head(10)

chromosome    78239
scaffold         59
Name: coord_system, dtype: int64

In [12]:
ensg.gene_df.gene_biotype.value_counts().head(10)

lncRNA                                32957
protein_coding                        21748
processed_pseudogene                   9329
TEC                                    3220
miRNA                                  2206
unprocessed_pseudogene                 2158
snoRNA                                 1507
snRNA                                  1381
transcribed_unprocessed_pseudogene      995
transcribed_processed_pseudogene        949
Name: gene_biotype, dtype: int64

In [13]:
pd.crosstab(ensg.gene_df.coord_system, ensg.gene_df.primary_assembly, margins=True)

primary_assembly,False,True,All
coord_system,,,
chromosome,0,78239,78239
scaffold,59,0,59
All,59,78239,78298


In [14]:
ensg.gene_df.mhc.value_counts()

Series([], Name: mhc, dtype: int64)

In [15]:
len(ensg.gene_df)

78298

## alternative gene alleles

Related:

- [OTP: Origin of genes_with_non_reference_ensembl_ids.tsv](https://github.com/opentargets/platform/issues/702)
- [biostars: map between different assemblies of one ensembl release](https://www.biostars.org/p/143956/)
- using `attrib_type.code = "non_ref"` for `primary_assembly` doesn't appear to return any results

In [16]:
ensg.representative_gene_df.head()

,rs_allele_group,ensembl_gene_id,gene_symbol,ensembl_created_date,seq_region,primary_assembly,alt_allele_group_id,alt_allele_attrib,alt_allele_is_representative,ensembl_representative_gene_id,is_representative_gene
0,0610005C13Rik,ENSMUSG00000109644,0610005C13Rik,2007-12-11 14:58:31,7,True,None,None,False,ENSMUSG00000109644,True
1,0610006L08Rik,ENSMUSG00000108652,0610006L08Rik,2015-07-06 11:47:18,7,True,None,None,False,ENSMUSG00000108652,True
2,0610009E02Rik,ENSMUSG00000086714,0610009E02Rik,2005-06-29 11:35:32,2,True,None,None,False,ENSMUSG00000086714,True
3,0610009L18Rik,ENSMUSG00000043644,0610009L18Rik,2004-08-20 10:27:42,11,True,None,None,False,ENSMUSG00000043644,True
4,0610010K14Rik,ENSMUSG00000020831,0610010K14Rik,2003-12-09 12:39:50,11,True,None,None,False,ENSMUSG00000020831,True


In [17]:
# looks like non_ref isn't set for human genes
query = '''
SELECT *
FROM gene_attrib
LEFT JOIN attrib_type
  ON gene_attrib.attrib_type_id = attrib_type.attrib_type_id
WHERE attrib_type.code = "non_ref"
LIMIT 5
'''
pd.read_sql(sql=query, con=ensg.connection_url)

,gene_id,attrib_type_id,value,attrib_type_id,code,name,description


In [18]:
ensg.representative_gene_df.alt_allele_attrib.value_counts()

Series([], Name: alt_allele_attrib, dtype: int64)

In [19]:
ensg.representative_gene_df.query("ensembl_gene_id != ensembl_representative_gene_id").head(2)

,rs_allele_group,ensembl_gene_id,gene_symbol,ensembl_created_date,seq_region,primary_assembly,alt_allele_group_id,alt_allele_attrib,alt_allele_is_representative,ensembl_representative_gene_id,is_representative_gene
76,1600029O15Rik,ENSMUSG00000121691,1600029O15Rik,2023-02-25 19:58:01,9,True,None,None,False,ENSMUSG00000057818,False
170,1700016A09Rik,ENSMUSG00000123653,1700016A09Rik,2024-05-06 19:10:14,8,True,None,None,False,ENSMUSG00000102737,False


# replaced ID converter

A single `old_stable_id` can map to multiple `new_stable_id`. For example, `ENSG00000152006`

https://uswest.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=AzhM62SpkvdiLC4H-6808613

Requested ID | Matched ID(s) | Releases
-- | -- | --
ENSG00000152006 | ENSG00000196273 | 26: ENSG00000196273.1
ENSG00000152006 | ENSG00000197016 | 26: ENSG00000197016.1
ENSG00000152006 | ENSG00000196239 | 26: ENSG00000196239.1

In [20]:
ensg.old_to_new_df.head(2)

,old_ensembl_gene_id,new_ensembl_gene_id
0,ENSMUSG00000037155,ENSMUSG00000044465
1,ENSMUSG00000000700,ENSMUSG00000066358


In [21]:
# some ensembl genes replaced by many new ensembl genes
ensg.old_to_new_df.old_ensembl_gene_id.value_counts().head(2)

ENSMUSG00000070062    131
ENSMUSG00000070181    131
Name: old_ensembl_gene_id, dtype: int64

In [22]:
# example
ensg._update_ensembl_gene("ENSG00000152006")

{'ENSG00000152006'}

In [23]:
ensg.old_to_newest_df.head(2)

,old_ensembl_gene_id,newest_ensembl_gene_id,is_current
0,ENSMUSG00000000700,ENSMUSG00000066358,False
1,ENSMUSG00000000700,ENSMUSG00000068941,False


In [24]:
len(ensg.old_to_newest_df)

71734

In [25]:
ensg.old_to_newest_df.is_current.value_counts()

True     58891
False    12843
Name: is_current, dtype: int64

## omni-updater

The omni-updater dataset is designed to convert ensembl gene IDs from input data to the current, representative ensembl_gene_ids for this ensembl release. It assumes:

- users want to update outdated genes with their replacements
- users want a dataset of representative genes only, and want to convert alternative alleles to representative genes

An inner join of a dataset with `update_df` on `input_ensembl_gene_id` will do the following:

- produce output ensembl_gene_ids that are current and representatives
- update outdated genes with their current identifiers. Outdated genes with no current replacement will be removed by the inner join.
- update alternative gene alleles with their representatives
- genes that are already represenative and current will map to themselves

In [26]:
ensg.update_df.head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
0,ENSMUSG00000000001,ENSMUSG00000000001,True,True,1,1
1,ENSMUSG00000000003,ENSMUSG00000000003,True,True,1,1


In [27]:
ensg.update_df.sort_values("input_maps_to_n_genes", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
32001,ENSMUSG00000075866,ENSMUSG00000119351,False,True,126,253
54448,ENSMUSG00000075866,ENSMUSG00000119496,False,True,126,15


In [28]:
ensg.update_df.sort_values("n_inputs_map_to_gene", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
39819,ENSMUSG00000076000,ENSMUSG00000119739,False,True,120,255
44002,ENSMUSG00000075977,ENSMUSG00000119864,False,True,126,255


In [29]:
(ensg.update_df.input_maps_to_n_genes == 1).mean()

0.5804328335362164

In [30]:
ensg.update_df.query("ensembl_gene_id == 'ENSG00000256263'")

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


In [31]:
print(
    f"The omni-updater contains {len(ensg.update_df):,} rows for mapping "
    f"{ensg.update_df.input_ensembl_gene_id.nunique():,} input genes to "
    f"{ensg.update_df.ensembl_gene_id.nunique():,} current, representative genes."
)

The omni-updater contains 137,189 rows for mapping 82,538 input genes to 77,981 current, representative genes.


In [32]:
# https://useast.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=P45VLMbogubpI0QA-6815464
ensg.update_df.query("input_ensembl_gene_id == 'ENSG00000201456'").head(3)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene


## cross-refrences (xrefs)

In [33]:
ensg.xref_df.head()

,ensembl_representative_gene_id,ensembl_gene_id,gene_symbol,xref_source,xref_accession,xref_label,xref_description,xref_info_type,xref_linkage_annotation,xref_curie
0,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,ArrayExpress,ENSMUSG00000000001,ENSMUSG00000000001,None,DIRECT,None,arrayexpress:ENSMUSG00000000001
1,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,EntrezGene,14679,Gnai3,G protein subunit alpha i3,DEPENDENT,None,ncbigene:14679
2,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,MGI,MGI:95773,Gnai3,G protein subunit alpha i3,DIRECT,None,mgi:95773
3,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,Reactome_gene,R-MMU-109582,R-MMU-109582,Hemostasis,DIRECT,None,reactome:R-MMU-109582
4,ENSMUSG00000000001,ENSMUSG00000000001,Gnai3,Reactome_gene,R-MMU-111885,R-MMU-111885,Opioid Signalling,DIRECT,None,reactome:R-MMU-111885


In [34]:
# datasets where there are ensembl_gene_id-xref_source-xref_accession pairs might not be distinct
xref_dup_df = ensg.xref_df[ensg.xref_df.duplicated(subset=["ensembl_gene_id", "xref_source", "xref_accession"], keep=False)]
xref_dup_df.xref_source.value_counts()

Series([], Name: xref_source, dtype: int64)

In [35]:
# xref sources versus info_types
df = pd.crosstab(ensg.xref_df.xref_source, ensg.xref_df.xref_info_type, margins=True)
df["bioregistry_prefix"] = df.index.to_series().replace(ensg._xref_prefix_updater).map(normalize_prefix)
df

xref_info_type,DEPENDENT,DIRECT,MISC,PROJECTION,All,bioregistry_prefix
xref_source,,,,,,
ArrayExpress,0,78298,0,0,78298,arrayexpress
EntrezGene,28531,0,0,0,28531,ncbigene
HGNC,0,0,0,3,3,hgnc
MGI,0,56128,0,0,56128,mgi
RFAM,0,2467,0,0,2467,rfam
Reactome_gene,0,88274,0,0,88274,reactome
Uniprot_gn,51465,0,0,0,51465,uniprot
WikiGene,28531,0,0,0,28531,wikigenes
miRBase,0,0,1360,0,1360,mirbase


## Gene Ontology xrefs

In [36]:
ensg.xref_go_df.head(3)

,ensembl_gene_id,go_id,go_label,go_evidence_codes,xref_info_types,xref_info_texts,ensembl_transcript_ids,ensembl_representative_gene_id
0,ENSMUSG00000000001,GO:0000139,Golgi membrane,IEA,PROJECTION,from homo_sapiens translation ENSP00000358867,ENSMUST00000000001,ENSMUSG00000000001
1,ENSMUSG00000000001,GO:0001664,G protein-coupled receptor binding,IBA,DIRECT,GO_Central,ENSMUST00000000001,ENSMUSG00000000001
2,ENSMUSG00000000001,GO:0003924,GTPase activity,"IBA,IEA,ISS,TAS","PROJECTION,DEPENDENT,DIRECT",",from homo_sapiens translation ENSP00000358867...",ENSMUST00000000001,ENSMUSG00000000001


In [37]:
# GO terms for CCR5
# compare to http://useast.ensembl.org/Homo_sapiens/Gene/Ontologies/molecular_function?g=ENSG00000160791
sorted(ensg.xref_go_df.query("ensembl_gene_id == 'ENSG00000160791'").go_label)

[]

## lrg xrefs

In [38]:
ensg.xref_lrg_df.head(2)

,ensembl_gene_id,lrg_gene_id


In [39]:
len(ensg.xref_lrg_df)

0

### ncbigene xrefs

In [40]:
ensg.xref_ncbigene_df.head()

,ensembl_representative_gene_id,ncbigene_id,gene_symbol,ncbigene_symbol
1,ENSMUSG00000000001,14679,Gnai3,Gnai3
26,ENSMUSG00000000003,54192,Pbsn,Pbsn
33,ENSMUSG00000000028,12544,Cdc45,Cdc45
51,ENSMUSG00000000031,14955,H19,H19
55,ENSMUSG00000000037,107815,Scml2,Scml2


In [41]:
# ensembl gene mapped to by multiple ncbigenes
ensg.xref_ncbigene_df.ensembl_representative_gene_id.value_counts().head(3)

ENSMUSG00000000001    1
ENSMUSG00000071531    1
ENSMUSG00000071573    1
Name: ensembl_representative_gene_id, dtype: int64

In [42]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ensembl_representative_gene_id.duplicated().sum()

(28204, 0)

In [43]:
# ncbigene mapped to by multiple ensembl genes, likely due to alt gene alleles
ensg.xref_ncbigene_df.ncbigene_id.value_counts().head(3)

108168560    7
102635519    4
22301        2
Name: ncbigene_id, dtype: int64

In [44]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ncbigene_id.duplicated().sum()

(28204, 28)

In [45]:
# ensg.xref_ncbigene_df.query("ensembl_representative_gene_id == 'ENSG00000231500'")
# ensg.xref_ncbigene_df.query("ncbigene_id == '51206'")

In [46]:
repr_ensembl_gene_ids = set(ensg.gene_df.ensembl_representative_gene_id)
len(repr_ensembl_gene_ids)

77981

In [47]:
# many of these genes should probably be alternative alleles rather than representative
ensg.gene_df.query("not primary_assembly and ensembl_gene_id==ensembl_representative_gene_id")

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
17497,ENSMUSG00000062783,3,Csprs,EntrezGene,114564,protein_coding,ensembl,2020-04-27 16:28:04,2020-08-25 18:37:28,GRCm39,...,111572,163011,-1,False,NaN,None,component of Sp100-rs,NCBI gene (formerly Entrezgene),114564,ENSMUSG00000062783
20614,ENSMUSG00000074720,2,ENSMUSG00000074720,None,None,protein_coding,ensembl,2020-04-27 16:31:13,2020-08-25 18:37:19,GRCm39,...,25923,27230,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000074720
22214,ENSMUSG00000079190,4,ENSMUSG00000079190,None,None,protein_coding,ensembl,2020-04-27 16:17:40,2020-08-25 18:37:25,GRCm39,...,167446,196478,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079190
22215,ENSMUSG00000079192,3,ENSMUSG00000079192,None,None,protein_coding,ensembl,2020-04-27 16:11:51,2020-08-25 18:37:27,GRCm39,...,123792,124928,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079192
22218,ENSMUSG00000079222,3,ENSMUSG00000079222,None,None,protein_coding,ensembl,2020-04-27 16:26:11,2020-08-25 18:37:27,GRCm39,...,17591,30203,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079222
22441,ENSMUSG00000079794,3,ENSMUSG00000079794,None,None,protein_coding,ensembl,2020-04-27 16:13:32,2020-08-25 18:37:27,GRCm39,...,135395,136519,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079794
22442,ENSMUSG00000079800,3,ENSMUSG00000079800,None,None,protein_coding,ensembl,2020-04-27 16:09:17,2020-08-25 18:37:26,GRCm39,...,9124,58882,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079800
22444,ENSMUSG00000079808,4,ENSMUSG00000079808,None,None,protein_coding,ensembl,2020-04-27 16:05:55,2020-08-25 18:37:18,GRCm39,...,128555,150452,-1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000079808
30066,ENSMUSG00000091585,3,ENSMUSG00000091585,None,None,protein_coding,ensembl,2020-04-27 17:39:52,2020-08-25 18:37:22,GRCm39,...,716296,720517,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000091585
31226,ENSMUSG00000094054,2,ENSMUSG00000094054,None,None,processed_pseudogene,ensembl,2020-04-27 16:15:58,2020-08-25 18:37:25,GRCm39,...,113868,114758,1,False,NaN,None,NaN,NaN,NaN,ENSMUSG00000094054
